In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../../LIB/')
from env import ENV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
X = pd.read_csv(ENV.POS_CASH_balance_ori.value)

In [3]:
class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)
        

In [4]:
def calculate_na(ser):
    return np.sum(ser.isnull())

def view_hist(ser):
    plt.hist(ser, range=(min(ser.values), max(ser.values)))
    
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [5]:
p = scan_nan_portion(X)
p = p.sort_values()
print(p.describe())
oe = ordinal_encoder()

count    8.000000
mean     0.000652
std      0.001207
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000652
max      0.002608
dtype: float64


# Data Clean

## 0 - 20

In [6]:
category_20 = []
failed = []
for each in p.iloc[:].index.values:
    
    print(X[each].describe())
    print('----')
    print(X[each].dtypes)
    num_values = len(X[each].value_counts())
    if num_values < 10:
        category_20.append(each)
        try:
            X[each] = X[each].astype('int')
        except ValueError:
            print('{} converted to int failed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'.format(each))
            failed.append(each)
    print('value counts: {}'.format(num_values))
    print('NA percentage: {}'.format(p[each]))
    print('========= End ==================')
print(failed)
# category_20.remove()

count    1.000136e+07
mean     1.903217e+06
std      5.358465e+05
min      1.000001e+06
25%      1.434405e+06
50%      1.896565e+06
75%      2.368963e+06
max      2.843499e+06
Name: SK_ID_PREV, dtype: float64
----
int64
value counts: 936325
NA percentage: 0.0
========= End ==================
count    1.000136e+07
mean     2.784039e+05
std      1.027637e+05
min      1.000010e+05
25%      1.895500e+05
50%      2.786540e+05
75%      3.674290e+05
max      4.562550e+05
Name: SK_ID_CURR, dtype: float64
----
int64
value counts: 337252
NA percentage: 0.0
========= End ==================
count    1.000136e+07
mean    -3.501259e+01
std      2.606657e+01
min     -9.600000e+01
25%     -5.400000e+01
50%     -2.800000e+01
75%     -1.300000e+01
max     -1.000000e+00
Name: MONTHS_BALANCE, dtype: float64
----
int64
value counts: 96
NA percentage: 0.0
========= End ==================
count     10001358
unique           9
top         Active
freq       9151119
Name: NAME_CONTRACT_STATUS, dtype: object
---

In [7]:
col = 'NAME_CONTRACT_STATUS'
X[col].value_counts()

Active                   9151119
Completed                 744883
Signed                     87260
Demand                      7065
Returned to the store       5461
Approved                    4917
Amortized debt               636
Canceled                      15
XNA                            2
Name: NAME_CONTRACT_STATUS, dtype: int64

### Ordinal Encoding

In [8]:
col = 'NAME_CONTRACT_STATUS'

oe.fit(X[col],NA_VALUE='XNA')
X[col] = oe.transform(X[col])
category_20.append(col)
###############################################



### FillNa

In [9]:
###############################################

col = 'CNT_INSTALMENT'
X[col] = X[col].fillna(int(X[col].mean()))

###############################################

col = 'CNT_INSTALMENT_FUTURE'
X[col] = X[col].fillna(int(X[col].mean()))



# Saving

In [10]:
X.to_pickle(ENV.POS_CASH_balance_clean.value)

In [11]:
np.sum(X.isnull())

SK_ID_PREV               0
SK_ID_CURR               0
MONTHS_BALANCE           0
CNT_INSTALMENT           0
CNT_INSTALMENT_FUTURE    0
NAME_CONTRACT_STATUS     0
SK_DPD                   0
SK_DPD_DEF               0
dtype: int64

In [13]:
categori_col = list(set(category_20))
categori_col

['NAME_CONTRACT_STATUS']

In [14]:
import pickle
categori_col_exist = pickle.load(open(ENV.clean_categorical_col.value,'rb'))
categori_col = list(set(categori_col + categori_col_exist))
pickle.dump(categori_col,open(ENV.clean_categorical_col.value,'wb'))